In [3]:
%%time
import pandas as pd
import numpy
from collections import Counter
from snownlp import SnowNLP
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

df = pd.read_csv('category_vaccine.csv', sep='|')

# ckiplab word segment (中研院斷詞)
ws = CkipWordSegmenter(level=2)
pos = CkipPosTagger(level=2)
ner = CkipNerChunker(level=2)


## Word Segmentation
tokens = ws(df.contents)

## POS
tokens_pos = pos(tokens)

## word pos pair 詞性關鍵字
word_pos_pair = [list(zip(w, p)) for w, p in zip(tokens, tokens_pos)]

## NER命名實體辨識
entity_list = ner(df.contents)

# Remove stop words and filter using POS tag (tokens_v2)
#with open('stops_chinese_traditional.txt', 'r', encoding='utf8') as f:
#    stops = f.read().split('\n')

# 過濾條件:兩個字以上 特定的詞性
allowPOS = ['Na', 'Nb', 'Nc', 'VA', 'VAC', 'VB', 'VC']

tokens_v2 = []
for wp in word_pos_pair:
    tokens_v2.append([w for w, p in wp if (len(w) >= 2) and p in allowPOS])

# Insert tokens into dataframe (新增斷詞資料欄位)
df['tokens'] = tokens
df['tokens_v2'] = tokens_v2
df['entities'] = entity_list
df['token_pos'] = word_pos_pair

# Calculate word count (frequency) 計算字頻(次數)
# allowPOS 過濾條件: 特定的詞性 兩個字以上
allowPOS = ['Na', 'Nb', 'Nc', 'VC']

def word_frequency(wp_pair):
    filtered_words = []
    for word, pos in wp_pair:
        if (pos in allowPOS) & (len(word) >= 2):
            filtered_words.append(word)
        #print('%s %s' % (word, pos))
    counter = Counter(filtered_words)
    return counter.most_common(200)


keyfreqs = []
for wp in word_pos_pair:
    topwords = word_frequency(wp)
    keyfreqs.append(topwords)

df['top_key_freq'] = keyfreqs

# Abstract (summary) and sentimental score(摘要與情緒分數)
summary = []
sentiment = []
for text in df.contents:
    sn = SnowNLP(text)
    summary.append(sn.summary())
    sentiment.append(round(sn.sentiments, 2))

df['summary'] = summary
df['sentiment'] = sentiment

# Rearrange the colmun order for readability
df = df[[
    'categories', 'titles', 'contents', 'sentimen', 'summary',
    'top_key_freq', 'tokens', 'tokens_v2', 'entities', 'token_pos', 'urls',
]]

# Save data to disk
df.to_csv('vaccine_preprocessed.csv', sep='|', index=False)

## Read it out 讀出看看
#df = pd.read_csv('cna_dataset_preprocessed.csv', sep='|')
#df.head(1)

print("process OK!")

Inference: 100%|██████████| 7/7 [15:19<00:00, 131.34s/it]


KeyError: "['category', 'title', 'content', 'link'] not in index"

In [7]:
# Rearrange the colmun order for readability
df = df[[
    'categories', 'titles', 'contents', 'sentiment', 'summary',
    'top_key_freq', 'tokens', 'tokens_v2', 'entities', 'token_pos', 'urls',
]]

# Save data to disk
df.to_csv('vaccine_preprocessed.csv', sep='|', index=False)

## Read it out 讀出看看
#df = pd.read_csv('cna_dataset_preprocessed.csv', sep='|')
#df.head(1)

print("process OK!")

process OK!


In [10]:
## Read it out 讀出看看
df = pd.read_csv('vaccine_preprocessed.csv', sep='|')
df.head(1)

,categories,titles,contents,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,urls
0,AZ,澎湖確診數升 兒童疫苗接種率已逾7成,澎湖COVID-19（2019冠狀病毒疾病）確診數攀升，澎湖縣政府今天公布新增182例，還有...,0.0,"['澎湖5至11歲兒童疫苗接種種率已逾7成', '澎湖縣政府今天公布新增182例確診案例',...","[('澎湖', 5), ('疫苗', 5), ('接種', 4), ('確診', 3), (...","['澎湖', 'COVID-19', '（', '2019', '冠狀', '病毒', '疾...","['澎湖', '冠狀', '病毒', '疾病', '確診數', '澎湖縣', '政府', '...","[NerToken(word='澎湖', ner='GPE', idx=(0, 2)), N...","[('澎湖', 'Nc'), ('COVID-19', 'FW'), ('（', 'PARE...",https://news.google.com/articles/CBMiMmh0dHBzO...
